# AI-WEEK-02-PDF_Query_LLM

## 1. AI를 이용해서 PDF 내용을 질문 해보기

### 1-1. 라이브러리 설치
- LangChain
- OpenAI
- TikToken
- PyMuPDF
- FAISS

In [ ]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install pymupdf
!pip install faiss-cpu

### 1-2. OpenAI API Key 설정

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "{OPEN_AI_KEY}"

### 1-3. PDF파일 조회를 위해 Google Drive 연동하기
- 테스트를 위한 PDF 파일은 [GitHub](https://github.daumkakao.com/aiweek/AI-WEEK-02-PDF_Query_LLM)에서 다운로드 가능합니다. 
- 다운로드 한 PDF 파일을 자신의 Google Drive 루트 Path에 올려주세요.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

### 1-4. PDF에서 텍스트 추출
- [PyMuPDF](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/pdf.html?highlight=PyMuPDF#using-pymupdf) : PDF 파일을 분석하고, 텍스트를 추출합니다.

In [5]:
from langchain.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader('/content/gdrive/My Drive/2023_GPT4All_Technical_Report.pdf')

In [6]:
pages = loader.load_and_split()
raw_text = ""
for page in pages:
    text = page.page_content
    raw_text += text

In [ ]:
raw_text[:200]

### 1-5. 텍스트 분할
- [CharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/character_text_splitter.html) : 텍스트를 분할합니다.
- 정보를 탐색하는 동안 토큰 크기 제한에 걸리지 않도록 텍스트를 분할해야 합니다.
- 전체 텍스트는 1,000 글자로 분할합니다.
- 텍스트 분할 시, 문맥 유지를 위해 200 글자는 겹치도록 분할합니다.

In [8]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

In [ ]:
texts[0]

In [ ]:
texts[1]

### 1-6. 텍스트 임베딩 및 벡터 저장소
- [OpenAIEmbeddings](https://python.langchain.com/en/latest/modules/models/text_embedding/examples/openai.html?highlight=OpenAIEmbeddings) : OpenAI 의 임베딩을 적용합니다.
- [FAISS](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html?highlight=FAISS) : 효율적인 벡터 유사도를 구합니다.


In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

docsearch = FAISS.from_texts(texts, OpenAIEmbeddings())

## 1-7. [Document Question Answering](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html)
- 질문과 유사도가 높은 여러 문서를 가져온 다음, LLM 을 통해 답변을 작성합니다.
- [LangChain LLMs](https://python.langchain.com/en/latest/modules/models/llms.html) - [OpenAI](https://python.langchain.com/en/latest/reference/modules/llms.html#langchain.llms.OpenAI)
- LangChain 의 [체이닝](https://python.langchain.com/en/latest/modules/chains/getting_started.html) 방식이 여기서 빛을 발휘합니다.

In [11]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

openai = OpenAI(temperature=0)
qa = RetrievalQA.from_chain_type(llm=openai, chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
query = "저자는 누구입니까?"
qa.run(query)

In [ ]:
query = "GPT4all 을 훈련시키는데 비용이 얼마나 들었나요?"
qa.run(query)

In [ ]:
query = "훈련한 데이터셋의 사이즈는 얼마나 됩니까?"
qa.run(query)

### 1-8. 응용 해보기
- 다른 PDF로 변경해서 질문을 해보세요.
- PyMuPDF가 아닌 다른 [PDF 파서](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/pdf.html?highlight=pdf)를 써보세요.
- FAISS 가 아닌 [다른 VectorStore](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html)를 써보세요.
- [다양한 체이닝 방식](https://python.langchain.com/en/latest/modules/chains/index_examples/question_answering.html?highlight=load_qa_chain)을 살펴보세요.